This notebook will pull from the FPL API the weekly results for each player and stack all the results into a single df

Update weekly

In [41]:
import pandas as pd
import numpy as np
import requests

In [42]:
top500 = pd.read_csv('top500.csv', index_col='id_')
teams = pd.read_csv('teams', index_col=0)

In [43]:
histories = {}

ids = top500.index

for id_ in ids:
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(id_) + '/'
    r = requests.get(url)
    player_data = r.json()
    player_history = player_data['history']
    
    histories[id_] = player_history

In [44]:
history_dfs = {}

for key in histories:
    df = pd.DataFrame(histories[key])
    df['gameweek'] = [i+1 for i in range((len(df.index)))]
    df['id_'] = key
    df['sum_points'] = [sum(df['total_points'].iloc[:i+1]) for i in range(len(df.index))]
    df['web_name'] = top500.loc[key]['web_name']
    df['team'] = top500['team'].loc[key]
    
    df['next_week_points'] = df['total_points'].drop(0).append(pd.Series(0)).reset_index(drop=True)
    
    history_dfs[key] = df
    
    
    

In [45]:
pd.set_option('display.max_columns', None)
history_dfs[215]

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,gameweek,id_,sum_points,web_name,team,next_week_points
0,1,1,37,1,37.4,215,8,0,0,10.2,37.8,2019-08-10T11:30:00Z,78,19,0,0,0,0,1,0,868463,5,0,27.0,7,0,0,0,95,False,0,1,215,7,De Bruyne,11,11
1,2,3,44,0,127.7,215,16,2,0,22.5,59.2,2019-08-17T16:30:00Z,90,17,0,0,0,0,2,0,995410,2,2,38.0,11,2049,81457,79408,95,True,0,2,215,18,De Bruyne,11,5
2,1,0,22,0,42.3,215,22,1,0,7.6,17.2,2019-08-25T13:00:00Z,90,3,0,0,0,0,3,0,1566970,3,1,16.0,5,460538,510105,49567,96,False,0,3,215,23,De Bruyne,11,13
3,1,2,41,1,26.7,215,37,0,1,11.4,59.8,2019-08-31T14:00:00Z,68,4,0,0,0,0,4,0,1914603,0,4,27.0,13,293070,373295,80225,97,True,0,4,215,36,De Bruyne,11,1
4,0,0,6,0,36.8,215,46,0,0,4.8,9.2,2019-09-14T16:30:00Z,33,14,0,0,0,0,5,0,2386410,2,3,2.0,1,399792,438722,38930,98,False,0,5,215,37,De Bruyne,11,17
5,2,3,66,1,109.4,215,57,0,1,23.8,88.4,2019-09-21T14:00:00Z,90,18,0,0,0,0,6,0,2431080,0,8,40.0,17,35783,135252,99469,98,True,0,6,215,54,De Bruyne,11,9
6,2,1,30,0,33.6,215,65,1,0,7.5,23.2,2019-09-28T16:30:00Z,79,8,0,0,0,0,7,0,2871266,3,1,18.0,9,388729,412503,23774,99,False,0,7,215,63,De Bruyne,11,0
7,0,0,0,0,0.0,215,75,0,0,0.0,0.0,2019-10-06T13:00:00Z,0,20,0,0,0,0,8,0,2169368,2,0,0.0,0,-653701,141393,795094,100,True,0,8,215,63,De Bruyne,11,3
8,0,0,8,1,34.8,215,84,0,0,6.4,10.0,2019-10-19T16:30:00Z,90,7,0,0,0,0,9,0,2319067,2,0,19.0,3,96039,270262,174223,100,False,0,9,215,66,De Bruyne,11,6
9,1,0,24,1,91.1,215,95,0,0,14.0,37.2,2019-10-26T11:30:00Z,75,2,0,0,0,0,10,0,2640790,0,3,12.0,6,297701,311175,13474,101,True,0,10,215,72,De Bruyne,11,2


In [46]:
top500histories = pd.concat(history_dfs.values(), ignore_index=True)
top500histories.shape

(8702, 37)

In [47]:
top500histories.to_csv('pre_top500histories.csv')

In [48]:
top500histories.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'gameweek', 'id_', 'sum_points', 'web_name',
       'team', 'next_week_points'],
      dtype='object')